In [1]:
import requests
import pandas as pd
import duckdb
import json
import datamart_profiler
import io
id="datamart.zenodo.3749559.e7fe094d-2f11-4814-ac1a-3326902e063c"
filename=id.replace(".","_").replace("-","_")+".parquet"
resp=requests.get('https://auctus.vida-nyu.org/api/v1/download/'+id)

In [2]:
df = pd.read_csv(io.BytesIO(resp.content))
df['total_cases'].sum()

21817074.0

In [3]:
# data=resp.content.decode()
# list=[x.split(',') for x in data.split('\n')]
# print(list[1])
# df=pd.DataFrame(list[1:],columns=list[0])


In [4]:
df.columns

Index(['country', 'date', 'total_cases', 'new_cases', 'total_deaths',
       'new_deaths', 'total_recovered', 'active_cases', 'servious_critical',
       'total_cases_1M_pop', 'total_deaths_1M_pop', 'total_tests',
       'tests_1M_pop'],
      dtype='object')

In [5]:
profiles=datamart_profiler.process_dataset(df)
for c in profiles['columns']:
    dtypes=[]
    dtypes.append(c['structural_type'].split('/')[-1])
    for s in c['semantic_types']:
        dtypes.append(s.split('/')[-1])
    print(c['name'],dtypes)

/home/aromston/software/anaconda3/envs/auctus/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/aromston/software/anaconda3/envs/auctus/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/aromston/software/anaconda3/envs/auctus/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/aromston/software/anaconda3/envs/auctus/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the va

country ['Text', 'Enumeration']
date ['Text', 'Enumeration', 'DateTime']
total_cases ['Integer']
new_cases ['Integer']
total_deaths ['Integer']
new_deaths ['Integer']
total_recovered ['Integer']
active_cases ['Integer']
servious_critical ['Integer']
total_cases_1M_pop ['Float']
total_deaths_1M_pop ['Float']
total_tests ['Integer']
tests_1M_pop ['Integer']


In [6]:
df.to_parquet(filename)
df.to_csv('test_file.csv')  

In [7]:
con=duckdb.connect("testdb.db")
con.sql(" describe select * from "+filename).fetchdf()

,column_name,column_type,null,key,default,extra
0,country,VARCHAR,YES,NaN,NaN,NaN
1,date,VARCHAR,YES,NaN,NaN,NaN
2,total_cases,DOUBLE,YES,NaN,NaN,NaN
3,new_cases,DOUBLE,YES,NaN,NaN,NaN
4,total_deaths,DOUBLE,YES,NaN,NaN,NaN
5,new_deaths,DOUBLE,YES,NaN,NaN,NaN
6,total_recovered,DOUBLE,YES,NaN,NaN,NaN
7,active_cases,DOUBLE,YES,NaN,NaN,NaN
8,servious_critical,DOUBLE,YES,NaN,NaN,NaN
9,total_cases_1M_pop,DOUBLE,YES,NaN,NaN,NaN


In [8]:
df_cpy=df.copy()
df_cpy
profiles=datamart_profiler.process_dataset(df_cpy)
for c in profiles['columns']:
    dtypes=[]
    dtypes.append(c['structural_type'].split('/')[-1])
    for s in c['semantic_types']:
        dtypes.append(s.split('/')[-1])
    try:
        if "DateTime" in dtypes:
            df_cpy[c['name']] = pd.to_datetime(df_cpy[c['name']], format='%Y-%m-%d')
        elif "Integer" in dtypes:
            print("converting to int")
            df_cpy[c['name']]=df_cpy[c['name']].astype(int)
        elif "Float" in dtypes:
            print("converting to float")
            df_cpy[c['name']]=df_cpy[c['name']].astype(float)
        elif "Text" in dtypes:
            df_cpy[c['name']]=df_cpy[c['name']].fillna("null").astype(str)
    except:
        pass


/home/aromston/software/anaconda3/envs/auctus/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/aromston/software/anaconda3/envs/auctus/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/aromston/software/anaconda3/envs/auctus/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/aromston/software/anaconda3/envs/auctus/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the va

converting to int
converting to int
converting to int
converting to int
converting to int
converting to int
converting to int
converting to float
converting to float
converting to int
converting to int


/home/aromston/software/anaconda3/envs/auctus/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [110]:
df_cpy.dtypes

country                        object
date                   datetime64[ns]
total_cases                     int64
new_cases                       int64
total_deaths                    int64
new_deaths                      int64
total_recovered                 int64
active_cases                    int64
servious_critical               int64
total_cases_1M_pop            float64
total_deaths_1M_pop           float64
total_tests                     int64
tests_1M_pop                    int64
dtype: object

In [111]:
df.dtypes

country                 object
date                    object
total_cases            float64
new_cases              float64
total_deaths           float64
new_deaths             float64
total_recovered        float64
active_cases           float64
servious_critical      float64
total_cases_1M_pop     float64
total_deaths_1M_pop    float64
total_tests            float64
tests_1M_pop           float64
dtype: object

In [112]:
col='tests_1M_pop'
df_cpy[col].sum()==df[col].sum()
print(df_cpy[col].sum())
print(df[col].sum())

9831991
9831992.0


In [117]:
" ALTER TABLE "+filename+" ALTER total_cases TYPE VARCHAR;"

' ALTER TABLE datamart_zenodo_3749559_e7fe094d_2f11_4814_ac1a_3326902e063c.parquet ALTER total_cases TYPE VARCHAR;'

In [128]:
con.sql("drop table test")


In [129]:
con.sql("drop table test")
con.sql("CREATE TABLE test AS SELECT * FROM "+filename)
con.sql("ALTER TABLE test ALTER total_cases TYPE VARCHAR;")
con.sql("COPY test TO 'my_table.parquet' (FORMAT 'parquet');")
con.sql("select * from read_parquet('my_table.parquet')")

CatalogException: Catalog Error: Table with name test does not exist!
Did you mean "information_schema.tables"?

In [ ]:

datamart.socrata.data-virginia-gov.4n9k-sf8f

update DATASET set date= (select CAST(STRPTIME(date,'%m/%d/%Y %I:%M:%S %p') as TIMESTAMP) as date from DATASET); alter table DATASET alter date type timestamp;